In [ ]:
import openai
import os

## Set API Keys

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Build the Chatbot

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# The vectorstore we'll be using
from langchain.vectorstores import FAISS, Chroma

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

# The easy document loader for text
from langchain.document_loaders import TextLoader

# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.prompts.prompt import PromptTemplate

# Chat specific components
from langchain.memory import ConversationBufferMemory

In [ ]:
# Build the model
model_version = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.3, model_name=model_version)

# Load data
loader = UnstructuredPDFLoader("../documents/wftda_rules.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Load our data into embeddings
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(texts, embeddings)

# Expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [ ]:
# Setup our template to prompt the model to be a roller derby referee
template = """
You are an expert referee for the sport Roller Derby. Your goal is to help the user understand
the rules of Roller Derby, as well as assist in adjuticating any disputes confusing situations
that might occur where the rules are unclear or tricky to apply. You are friendly and helpful,
but also kind of sassy, because this is Roller Derby after all.
Please give references to the rulebook when possible.

{chat_history}
Human: {human_input}
Autoref:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", ai_prefix="Autoref")

In [ ]:
# Create a chain to answer questions
qa = ConversationalRetrievalChain.from_llm(llm, 
                                           retriever,
                                           memory=memory,
                                           prompt=prompt)

In [ ]:
chat_history = []
query = "Hi, I'm a new referee, what are some of the things that are most challenging for new referees?"
result = qa({""})